In [15]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
from IPython.display import display, HTML
import seaborn as sns
import random
import matplotlib.pyplot as plt
import plotly.express as px
import pandas_profiling as pp
from scipy import stats
from scipy.stats import norm
%matplotlib inline
import h2o
from h2o.automl import H2OAutoML

In [19]:
df = pd.read_csv('data2/landmarks.csv')
train, validate, test = np.split(df, [int(.7*len(df)), int(.85*len(df))])

In [21]:
train

,0,1,2,3,4,5,6,7,8,9,...,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404
0,0.506777,0.685168,-0.081059,0.508899,0.589536,-0.155494,0.507721,0.619720,-0.081626,0.481201,...,0.547818,0.421041,-0.017888,0.701051,0.395390,0.041529,0.714997,0.388807,0.042386,contempt
1,0.522592,0.728219,-0.069704,0.563158,0.622856,-0.154344,0.532915,0.651415,-0.075771,0.527397,...,0.563678,0.440509,-0.014150,0.693491,0.415474,0.097632,0.706497,0.406117,0.102601,contempt
2,0.507426,0.711682,-0.081520,0.490157,0.622659,-0.162877,0.500488,0.650763,-0.086197,0.465061,...,0.549762,0.444901,-0.025063,0.724568,0.406819,0.017024,0.740692,0.396217,0.016287,contempt
3,0.484002,0.716097,-0.095621,0.490276,0.590272,-0.168015,0.489010,0.631962,-0.094333,0.468822,...,0.548870,0.438363,-0.009491,0.714356,0.434477,0.052216,0.732640,0.424303,0.053430,contempt
4,0.534178,0.709634,-0.044477,0.526397,0.637113,-0.133007,0.526363,0.654088,-0.060292,0.489256,...,0.537986,0.445891,-0.039478,0.672400,0.397646,0.011078,0.682771,0.388529,0.011500,contempt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23888,0.473343,0.647458,-0.020685,0.530409,0.584438,-0.092517,0.488883,0.591007,-0.034667,0.512971,...,0.533296,0.436306,-0.027793,0.608998,0.415199,0.075503,0.620629,0.402990,0.081695,sad
23889,0.538998,0.730614,-0.039409,0.575328,0.637325,-0.125219,0.543802,0.658993,-0.053053,0.533452,...,0.560976,0.431589,-0.032142,0.673303,0.383008,0.057707,0.684204,0.371329,0.060776,sad
23890,0.507124,0.710650,-0.070015,0.531728,0.612457,-0.140365,0.515420,0.643435,-0.071740,0.506756,...,0.560545,0.463939,-0.013874,0.696620,0.451923,0.066751,0.709731,0.436272,0.071047,sad
23891,0.486359,0.700483,-0.081752,0.473179,0.611061,-0.144204,0.483137,0.640749,-0.077841,0.453750,...,0.535396,0.442426,-0.016405,0.693518,0.411711,0.017808,0.708412,0.401323,0.016646,sad


In [18]:
h2o.init(
    nthreads=8,     # number of threads when launching a new H2O server
    max_mem_size=8  # in gigabytes
)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.8.1" 2023-08-24; OpenJDK Runtime Environment (build 17.0.8.1+1-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 17.0.8.1+1-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /home/vorkov/Workspace/Emotion-Decetion-Service/venv/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpvi9y9xzy
  JVM stdout: /tmp/tmpvi9y9xzy/h2o_vorkov_started_from_python.out
  JVM stderr: /tmp/tmpvi9y9xzy/h2o_vorkov_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,"14 days, 1 hour and 20 minutes"
H2O_cluster_name:,H2O_from_python_vorkov_97l4u7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [ ]:
train_h2o_frame = h2o.H2OFrame(train)
test_h2o_frame = h2o.H2OFrame(test)

x = train_h2o_frame.columns
y = "1404"
x.remove(y)

aml = H2OAutoML(max_models=10, seed=1)
aml.train(x=x, y=y, training_frame=train_h2o_frame)

 (cancelled) 100%



KeyboardInterrupt



In [24]:
aml.distribution

'AUTO'